# Assignment 2

Deadline: 26.03.2025, 12:00 CET

<Add your name, student-id and emal address>

- Maxmilian Krägeloh, 23-754-088, maximilianwerner.kraegeloh@uzh.ch

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [ ]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()
src_path = os.getenv('PROJECT_SOURCE_DIR')
#print(src_path)
sys.path.append(src_path)

from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [4]:
def linearize_turnover_constraint(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        # Dimensions
        n = len(self.problem_data.get('q'))
        m = 0 if self.problem_data.get('G') is None else self.problem_data.get('G').shape[0]

        # New dimension will be 2n (original weights + absolute change variables)
        n_new = 2 * n

        if isinstance(x_init, pd.Series):
            x_init = x_init.values

        # Update the coefficients of the objective function
        # P matrix: pad with zeros for new variables
        P_old = self.problem_data.get('P')
        P = np.zeros((n_new, n_new))
        if P_old is not None:
            P[:n, :n] = P_old

        # q vector: pad with zeros (no penalty on auxiliary variables in objective)
        q_old = self.problem_data.get('q')
        q = np.zeros(n_new)
        if q_old is not None:
            q[:n] = q_old


        # Update the equality constraints
        A_old = self.problem_data.get('A')
        if A_old is not None:
            A = np.zeros((A_old.shape[0], n_new))
            A[:, :n] = A_old
        else:
            A = None

        # Update the inequality constraints
        # Existing constraints
        G_old = self.problem_data.get('G')
        if G_old is not None:
            G_base = np.zeros((m, n_new))
            G_base[:, :n] = G_old
        else:
            G_base = np.zeros((0, n_new))

        # Add turnover constraints: 
        # 1. t_i >= w_i - x_init_i
        # 2. t_i >= -(w_i - x_init_i)
        # 3. sum(t_i) <= to_budget
        G_turnover = np.zeros((2*n + 1, n_new))
        h_turnover = np.zeros(2*n + 1)
        
        for i in range(n):
            # t_i >= w_i - x_init_i
            G_turnover[i, i] = -1      # w_i
            G_turnover[i, n + i] = 1   # t_i
            h_turnover[i] = -x_init[i]
            
            # t_i >= -(w_i - x_init_i)
            G_turnover[n + i, i] = 1   # w_i
            G_turnover[n + i, n + i] = 1  # t_i
            h_turnover[n + i] = x_init[i]
        
        # Total turnover constraint
        G_turnover[2*n, n:] = 1  # sum of t_i
        h_turnover[2*n] = to_budget
        
        # Combine all inequality constraints
        G = np.vstack((G_base, G_turnover))
        h_old = self.problem_data.get('h')
        h = np.concatenate((h_old if h_old is not None else np.array([]), h_turnover))

        # Update lower and upper bounds
        lb_old = self.problem_data.get('lb')
        ub_old = self.problem_data.get('ub')
        
        lb = np.zeros(n_new)
        ub = np.zeros(n_new)
        
        # Original variables keep their bounds
        if lb_old is not None:
            lb[:n] = lb_old
        else:
            lb[:n] = -np.inf
            
        if ub_old is not None:
            ub[:n] = ub_old
        else:
            ub[:n] = np.inf
        
        # Auxiliary variables (t_i) are non-negative and unbounded above
        lb[n:] = 0
        ub[n:] = np.inf

        # Override the original matrices (notice: b does not change)
        self.update_problem_data({
            'P': P,
            'q': q,
            'G': G,
            'h': h,
            'A': A,
            'lb': lb,
            'ub': ub
        })

        return None

## Demo

#### Create P and q

In [5]:
# Load the msci country index data
N = 10
data = load_data_msci(path = '../data/', n = N)
X = data['return_series']

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)

print("Vector of expected returns (q):") #mean return of each asset over time
print(q)

print("\nCovariance matrix (P):") #covariance matrix (how assets move together) -> isnt exactly the same as created due to randomness
print(P)

Vector of expected returns (q):
AT    0.000130
AU    0.000288
BE    0.000047
CA    0.000269
CH    0.000149
DE    0.000151
DK    0.000429
ES    0.000128
FI    0.000145
FR    0.000199
dtype: float64

Covariance matrix (P):
          AT        AU        BE        CA        CH        DE        DK  \
AT  0.000239  0.000054  0.000125  0.000075  0.000097  0.000138  0.000097   
AU  0.000054  0.000104  0.000039  0.000030  0.000035  0.000041  0.000041   
BE  0.000125  0.000039  0.000175  0.000064  0.000104  0.000137  0.000093   
CA  0.000075  0.000030  0.000064  0.000130  0.000058  0.000087  0.000053   
CH  0.000097  0.000035  0.000104  0.000058  0.000120  0.000121  0.000086   
DE  0.000138  0.000041  0.000137  0.000087  0.000121  0.000202  0.000105   
DK  0.000097  0.000041  0.000093  0.000053  0.000086  0.000105  0.000151   
ES  0.000150  0.000044  0.000138  0.000081  0.000116  0.000164  0.000100   
FI  0.000140  0.000050  0.000136  0.000091  0.000126  0.000180  0.000119   
FR  0.000143  0.000

### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance.

In [6]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
#constraints.add_box(lower=0.05, upper=0.15)  # Force 5-15% per asset
GhAb = constraints.to_GhAb()

print(constraints.box)
print(constraints.budget)

# Create a quadratic program and linearize the turnover constraint
qp = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0, # * 0 : pure risk minimization problem  // -q : pure return maximization problem // * 0.00001 add tiny return bias
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the linearized turnover constraint method to the instance of class QuadraticProgram
qp.linearize_turnover_constraint = types.MethodType(linearize_turnover_constraint, qp)


{'box_type': 'LongOnly', 'lower': AT    0.0
AU    0.0
BE    0.0
CA    0.0
CH    0.0
DE    0.0
DK    0.0
ES    0.0
FI    0.0
FR    0.0
dtype: float64, 'upper': AT    1.0
AU    1.0
BE    1.0
CA    1.0
CH    1.0
DE    1.0
DK    1.0
ES    1.0
FI    1.0
FR    1.0
dtype: float64}
{'Amat': AT    1.0
AU    1.0
BE    1.0
CA    1.0
CH    1.0
DE    1.0
DK    1.0
ES    1.0
FI    1.0
FR    1.0
dtype: float64, 'sense': '=', 'rhs': 1}


### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected.

In [7]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)
print("Initial portfolio weights:") 
print(x_init)

initial_objective = qp.objective_value(x=x_init)

# Add the linearized turnover constraint
qp.linearize_turnover_constraint(x_init=x_init, to_budget=0.5)

# Solve the problem
qp.solve()

# Check the turnover
solution = qp.results.get('solution')
ids = constraints.ids
weights = pd.Series(solution.x[:len(ids)], index=ids)

# After solving
print(f"Found solution: {solution.found}")
print(f"Status: {solution.extras['status']}")

#print("Solution:")
#print(solution.x)

final_objective = qp.objective_value()

print(f"Objective value before optimization: {initial_objective:.6f}")
print(f"Objective value after optimization: {final_objective:.6f}")

print("Weights after: ")
print(weights)

print("Turnover:")
print(np.abs(weights - x_init).sum())



Initial portfolio weights:
AT    0.1
AU    0.1
BE    0.1
CA    0.1
CH    0.1
DE    0.1
DK    0.1
ES    0.1
FI    0.1
FR    0.1
dtype: float64
Found solution: True
Status: optimal
Objective value before optimization: 0.000056
Objective value after optimization: 0.000056
Weights after: 
AT    0.1
AU    0.1
BE    0.1
CA    0.1
CH    0.1
DE    0.1
DK    0.1
ES    0.1
FI    0.1
FR    0.1
dtype: float64
Turnover:
0.0


In [8]:
print(np.allclose(P, P.T))

from numpy.linalg import eigvalsh
print("Min eigenvalue: ", eigvalsh(P).min())  # Should be > 0

equal_weights = np.full(10, 0.1)
variance = equal_weights.T @ P @ equal_weights  # Should match your "objective value before optimization"
print("Variance: ", variance)

perturbed_weights = equal_weights + np.random.normal(0, 0.01, 10)
perturbed_weights /= perturbed_weights.sum()
print("Perturbed variance: ", perturbed_weights.T @ P @ perturbed_weights) # If this is larger, your equal weights are indeed optimal.

True
Min eigenvalue:  1.4405363387419722e-05
Variance:  0.00011201261073643869
Perturbed variance:  0.00011406173731229363
